In [1]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='localhost', username='cgoeldel', password='Goe1409ldel')



In [2]:
import polars as pl
from IPython.display import display

batch_size = 1
neighbourhood_size = 10







def get_batch(mode):
    offset = 0. if mode == "train" else 0.9
    sample = 0.9 if mode == "train" else 0.1
    sites =  client.query_arrow(f"with sites as (select * except (trinucleotide_context, pedigree, id) from methylome sample {sample} offset {offset} where generation = 0 order by rand() limit {batch_size * 10}) select *, (location - start) as start_diff, (location - end) as end_diff from sites left join genes on sites.chromosome = genes.chromosome and sites.strand = genes.strand where sites.location between genes.start and genes.end limit {batch_size}")

    df = pl.DataFrame(sites)


    df = df.drop(["name", "genes.chromosome", "genes.strand"]) # What about absolute Start and End?


    display(df)

    for site in df.iter_rows(named=True):
        # Get each site in all generations and lines
        all_generations_and_neighbours = client.query_arrow(f"select * except (trinucleotide_context, pedigree, id) from methylome where chromosome = {site['chromosome']} and location between {site['location'] - neighbourhood_size }  and {site['location'] + neighbourhood_size}  and strand = {site['strand']} order by generation, line, location")
        g = pl.DataFrame(all_generations_and_neighbours)
        display(g)

       
        # Get surrounding histone mods 
        histone_mods = client.query_arrow(f"select *, ({site['location']} - start ) as start_diff, ({site['location']} -end) as end_diff from histone_mods where chromosome = {site['chromosome']} and (start between {site['location'] - neighbourhood_size }  and {site['location'] + neighbourhood_size} or end between {site['location'] - neighbourhood_size }  and {site['location'] + neighbourhood_size}) ")
        h = pl.DataFrame(histone_mods)
        display(h)
        # Get surrounding chromatine states
        chr_states = client.query_arrow(f"select *, ({site['location']} - start ) as start_diff, ({site['location']} -end) as end_diff from chr_states where chromosome = {site['chromosome']} and (start between {site['location'] - neighbourhood_size }  and {site['location'] + neighbourhood_size} or end between {site['location'] - neighbourhood_size }  and {site['location'] + neighbourhood_size}) ")
        c = pl.DataFrame(chr_states)
        display(c)

        
        


# sites_and_genes = client.query_arrow("with sites as (select * except trinucleotide_context, pedigree, id from methylome where generation = 0 order by rand() limit 10) select * from sites left join genes on sites.chromosome = genes.chromosome and sites.strand = genes.strand where sites.location between genes.start and genes.end")
# pl.DataFrame(sites_and_genes)


get_batch("train")

{'chromosome': 3, 'location': 5856282, 'strand': 2, 'context': 3, 'count_methylated': 0, 'count_total': 14, 'posteriormax': 0.9998999834060669, 'status': 1, 'meth_lvl': 0.0017000000225380063, 'generation': 0, 'line': 0, 'start': 5855632, 'end': 5859189, 'type': 2, 'start_diff': 650, 'end_diff': -2907}


chromosome,location,strand,context,count_methylated,count_total,posteriormax,status,meth_lvl,generation,line
u8,u32,i8,i8,u8,u8,f32,i8,f32,u8,u8
3,5856281,2,3,0,14,0.9999,1,0.0017,0,0
3,5856282,2,3,0,14,0.9999,1,0.0017,0,0
3,5856281,2,3,0,16,0.9999,1,0.0014,1,2
3,5856282,2,3,1,15,0.9997,1,0.0014,1,2
3,5856281,2,3,0,9,0.9999,1,0.0014,1,8
3,5856282,2,3,0,10,0.9999,1,0.0014,1,8
3,5856281,2,3,0,16,0.9999,1,0.0014,2,2
3,5856282,2,3,0,16,0.9999,1,0.0014,2,2
3,5856281,2,3,0,17,0.9999,1,0.0015,2,8


chromosome,start,end,modification,start_diff,end_diff
u8,u32,u32,str,i64,i64
3,5856269,5856274,"""H3K27Me3""",13,8
3,5856274,5856275,"""H3K27Me3""",8,7
3,5856275,5856277,"""H3K27Me3""",7,5
3,5856277,5856279,"""H3K27Me3""",5,3
3,5856279,5856282,"""H3K27Me3""",3,0
3,5856282,5856285,"""H3K27Me3""",0,-3
3,5856285,5856288,"""H3K27Me3""",-3,-6
3,5856288,5856291,"""H3K27Me3""",-6,-9
3,5856291,5856292,"""H3K27Me3""",-9,-10


chromosome,start,end,state,start_diff,end_diff
u8,u32,u32,u8,i64,i64


In [3]:
import polars as pl
import pyarrow as pa

data_sizes = client.query_arrow("Select count(*) as samples,chromosome, generation, line, pedigree from methylome_wt group by chromosome, pedigree, generation, line order by chromosome, generation, line")
num_chromosomes = len(data_sizes['chromosome'].unique())

data_sizes = pl.DataFrame(data_sizes)
data_sizes

samples,chromosome,generation,line,pedigree
u64,u8,u8,u8,str
10856447,1,0,0,"""MA3"""
10856447,1,1,2,"""MA3"""
10856447,1,1,8,"""MA3"""
10856447,1,2,2,"""MA3"""
10856447,1,2,8,"""MA3"""
10856447,1,4,2,"""MA3"""
10856447,1,4,8,"""MA3"""
10856447,1,5,2,"""MA3"""
10856447,1,5,8,"""MA3"""


In [ ]:
samples_per_chromosome = data_sizes.group_by("chromosome").agg(pl.col("samples").min())
samples_per_chromosome =  samples_per_chromosome.to_struct("samples")

samples_per_chromosome.to_list()
samples_per_chromosome = {item['chromosome']:item['samples'] for item in samples_per_chromosome.to_list()}
samples_per_chromosome

{4: 6727432, 5: 9690976, 2: 7063707, 1: 10856447, 3: 8520954}

In [ ]:
import torch
pedigree_dict = {
    'MA3': 0,
    'CMT3': 1,
    'SUV456': 2,
    'ROS': 3,
    'NRPE': 4
}

trinucleotide_context = {
'CTC': 0, 
 'CGA': 1,
 'CCC':  2,
 'CAT':  3,
 'CCG':  4,
 'CTG':  5,
 'CAA':  6,
 'CGT':  7,
 'CTT':  8,
 'CGG':  9,
 'CGN':  10,
 'CAC':  11,
 'CCT':  12,
 'CTA':  13,
 'CGS':  14,
 'CAG':  15,
 'CGC':  16,
 'CGK':  17,
 'CCA': 18,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def encode_methylome(df): 
    df = df.with_columns(
        pl.col("pedigree").map_dict(pedigree_dict).alias("pedigree"),
        pl.col("trinucelotide_context").map_dict(trinucleotide_context).alias("trinucelotide_context"),
    )

    data = torch.tensor(df.to_numpy(), dtype=torch.float32 ,device=device)

    return data
    # torch.tensor(df.to_numpy, dtype=torch.float32 ,device=device)

batch_size = 6 * num_chromosomes # 6 samples per chromosome = 30 samples per batch
block_size = 8

def get_batch(context): 
    for i in range(num_chromosomes):

        idx = torch.randint(0, samples_per_chromosome[i] - block_size, (batch_size,))

    
        methylome_first_batch = client.query_arrow("select * from methylome_wt where location between 0 and 100000 limit 1000")
    

torch.Size([10000, 13])

In [ ]:
chromosome_lengths = 

5